In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [2]:
!echo $CUDA_HOME

In [3]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" pip install llama-cpp-python --upgrade --force-reinstall --no-cache-dir

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 MB 211.5 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 118.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 169.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.3/133.3 kB 216.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 235.2 MB/s eta 0:00:00a 0:00:01
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.77-cp310-cp310-linux_x86_64.whl size=144358573 sha256=231b678c53e8087413615beeb170330cc4e746eb5af7ba6cb2ddb1e893485853
  Stored in directory: /tmp/pip-ephem-wheel-cache-gu0jierl/wheels/ed/55/a1/6d6c2ef6fed3ef054b4170d8bcd05a09e6dc971db7fad955ff
Successfully built llama-cpp-python
  Attempting uninstall: typing-extensions
    Found 

In [4]:
!pip install llama-index-llms-llama-cpp -q


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip


In [19]:
from llama_index.llms.llama_cpp import LlamaCPP

def messages_to_prompt(messages):
    prompt = ""
    for message in messages:
        if message.role == 'system':
         prompt += f"<|system|>Chenosis is a cross-industry API marketplace that enables developers, entrepreneurs and businesses to integrate into the fastest-growing library of open APIs. strictly answer in around two lines.\n{message.content}</s>\n"
        elif message.role == 'user':
         prompt += f"<|user|>\n{message.content}</s>\n"
        elif message.role == 'assistant':
         prompt += f"<|assistant|>\n{message.content}</s>\n"

    # ensure we start with a system prompt, insert blank if needed
    if not prompt.startswith("<|system|>\n"):
        prompt = "<|system|>Chenosis is a cross-industry API marketplace that enables developers, entrepreneurs and businesses to integrate into the fastest-growing library of open APIs. strictly answer in around two lines.\n</s>\n" + prompt

    # add final assistant prompt
    prompt = prompt + "<|assistant|>\n"

    return prompt

def completion_to_prompt(completion):
    return f"<|system|>\n</s>\n<|user|>\n{completion}</s>\n<|assistant|>\n"

model_url = "https://huggingface.co/TheBloke/zephyr-7B-beta-GGUF/resolve/main/zephyr-7b-beta.Q4_0.gguf"

llm = LlamaCPP(
    model_url=model_url,
    temperature=0.1,
    max_new_tokens=150,
    context_window=7500,
    generate_kwargs={},
    model_kwargs={"n_gpu_layers": -1},  # if compiled to use GPU
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=True,
)

response = llm.complete("Hello, how are you?")
print(str(response))

total size (MB): 4108.92


3919it [00:16, 242.86it/s]                          
llama_model_loader: loaded meta data with 21 key-value pairs and 291 tensors from /tmp/llama_index/models/zephyr-7b-beta.Q4_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = huggingfaceh4_zephyr-7b-beta
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32        

ValueError: Failed to load model from file: /tmp/llama_index/models/zephyr-7b-beta.Q4_0.gguf

In [2]:
response = llm.complete("what is the capital of india?")
print(str(response))

Llama.generate: prefix-match hit

llama_print_timings:        load time =     838.01 ms
llama_print_timings:      sample time =      87.20 ms /    89 runs   (    0.98 ms per token,  1020.69 tokens per second)
llama_print_timings: prompt eval time =      76.56 ms /    18 tokens (    4.25 ms per token,   235.09 tokens per second)
llama_print_timings:        eval time =    1225.04 ms /    88 runs   (   13.92 ms per token,    71.83 tokens per second)
llama_print_timings:       total time =    1424.23 ms /   106 tokens


The capital city of India is New Delhi. It serves as the seat of the federal government, parliament, and supreme court of India. However, New Delhi is a territory within the National Capital Territory of Delhi (NCT), which also includes other cities like Delhi, Noida, Gurgaon, and Faridabad. The largest city by population in India is Mumbai (formerly known as Bombay).


In [9]:
response_iter = llm.stream_complete("Can you write me a poem about fast cars?")
for response in response_iter:
    print(response.delta, end="", flush=True)

Sleek and swift, they

Llama.generate: prefix-match hit


 glide along the track,
A symphony of metal and machine.
Their engines roar, a primal pack,
As they race towards their destiny unseen.

Fast cars, they're more than just a ride,
They're a passion, a dream, a need.
The wind in hair, the rush inside,
A thrill that's hard to heed.

They're born to run, to push the limits,
To test the laws of physics and time.
Their speed is measured by the sands of grits,
As they leave the world behind.

Fast cars, they're a work


llama_print_timings:        load time =     885.11 ms
llama_print_timings:      sample time =      51.01 ms /   150 runs   (    0.34 ms per token,  2940.54 tokens per second)
llama_print_timings: prompt eval time =      95.04 ms /    20 tokens (    4.75 ms per token,   210.45 tokens per second)
llama_print_timings:        eval time =    2318.93 ms /   149 runs   (   15.56 ms per token,    64.25 tokens per second)
llama_print_timings:       total time =    2666.80 ms /   169 tokens


In [10]:
!pip install llama-index llama-index-embeddings-huggingface llama-parse llama-index-readers-web -q


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip


In [4]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.llms.llama_cpp import LlamaCPP
from llama_parse import LlamaParse
from llama_index.core import SimpleDirectoryReader
from llama_index.llms.llama_cpp.llama_utils import (
    messages_to_prompt,
    completion_to_prompt,
)

In [5]:
# use Huggingface embeddings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [6]:
from llama_index.core import StorageContext, load_index_from_storage

# rebuild storage context
storage_context = StorageContext.from_defaults(persist_dir="chenosis")

# load index
index = load_index_from_storage(storage_context, embed_model=embed_model)

In [12]:
# set up query engine
query_engine = index.as_query_engine(llm=llm,similarity_top_k=5,streaming=True)

In [18]:
response_iter = query_engine.query("What is Chenosis")
for response in response_iter.response_gen:
    print(response, end="", flush=True)

Llama.generate: prefix-match hit


Chenosis is a cross-industry API marketplace that enables developers, entrepreneurs, and businesses to integrate into the fastest-growing library of open APIs. (Context information provided in the first query.)


llama_print_timings:        load time =     838.01 ms
llama_print_timings:      sample time =      16.23 ms /    45 runs   (    0.36 ms per token,  2771.79 tokens per second)
llama_print_timings: prompt eval time =    1135.94 ms /  2127 tokens (    0.53 ms per token,  1872.45 tokens per second)
llama_print_timings:        eval time =     663.35 ms /    44 runs   (   15.08 ms per token,    66.33 tokens per second)
llama_print_timings:       total time =    1872.97 ms /  2171 tokens


In [15]:
response_iter = query_engine.query("2. How would I prevent fraud?")
for response in response_iter.response_gen:
    print(response, end="", flush=True)

Llama.generate: prefix-match hit


One solution to prevent fraud is to implement the Mobile Identity Fraud Bundle offered by Chenosis. This bundle includes a selection of APIs designed to detect fraud and authenticate users in real time, such as SIM Swap, KYC Verify, and s-MNV. By using these APIs, you can validate your customers' identities, compare their given information with what is stored on the MTN database, and detect suspicious activity before it becomes a problem. This will help prioritize efficient security procedures within a budget and reduce your risk of mobile identity fraud.


llama_print_timings:        load time =     838.01 ms
llama_print_timings:      sample time =      44.14 ms /   119 runs   (    0.37 ms per token,  2695.85 tokens per second)
llama_print_timings: prompt eval time =    2678.07 ms /  4510 tokens (    0.59 ms per token,  1684.05 tokens per second)
llama_print_timings:        eval time =    1942.79 ms /   118 runs   (   16.46 ms per token,    60.74 tokens per second)
llama_print_timings:       total time =    4852.32 ms /  4628 tokens


In [16]:
response_iter = query_engine.query("What is Radify?")
for response in response_iter.response_gen:
    print(response, end="", flush=True)

Llama.generate: prefix-match hit


Radify is a smart diagnostic solution that uses AI technology to provide fast and informed diagnoses for chest scans. It can identify common illnesses such as TB, malaria, pneumonia, and COVID-19, and deliver results within a minute. Radify is designed to support medical practitioners in making better decisions by providing them with expert opinions and recommendations, especially in regions where there is a shortage of radiologists. It aims to bridge the gap of access to enough expert radiologists and relieve the pressure on in-house radiologists, allowing them to focus on critical cases. Radify is part of Chenosis, a company that provides open access to fast, affordable, and informed medical care through smart technology solutions.


llama_print_timings:        load time =     838.01 ms
llama_print_timings:      sample time =      55.04 ms /   150 runs   (    0.37 ms per token,  2725.39 tokens per second)
llama_print_timings: prompt eval time =    2310.78 ms /  4035 tokens (    0.57 ms per token,  1746.17 tokens per second)
llama_print_timings:        eval time =    2441.07 ms /   149 runs   (   16.38 ms per token,    61.04 tokens per second)
llama_print_timings:       total time =    5008.99 ms /  4184 tokens


In [20]:
response_iter = query_engine.query("5. What does this have to do with MTN?")
for response in response_iter.response_gen:
    print(response, end="", flush=True)

Llama.generate: prefix-match hit


The provided text material is a collection of use case stories, brochure copy, infographic scripts, and product announcements for various MTN API products, including Mobile Network Info, SIM Swap, KYC Verify, and s-MNV (silent Mobile Number Verification). These tools are designed to enhance the user experience by providing additional features such as location verification, fraud prevention, authentication, and identity verification. By leveraging these APIs, businesses can improve their customer onboarding processes, reduce friction in transactions, and provide a more seamless and secure experience for their users. Ultimately, this benefits MTN's customers by enabling them to offer better services and products to their own end-users.


llama_print_timings:        load time =     838.01 ms
llama_print_timings:      sample time =      57.62 ms /   150 runs   (    0.38 ms per token,  2603.35 tokens per second)
llama_print_timings: prompt eval time =    3493.38 ms /  5484 tokens (    0.64 ms per token,  1569.82 tokens per second)
llama_print_timings:        eval time =    2581.21 ms /   149 runs   (   17.32 ms per token,    57.72 tokens per second)
llama_print_timings:       total time =    6404.00 ms /  5633 tokens
